In [1]:
import tensorflow as tf
import keras
from keras import backend as K
import numpy as np
import re
from keras import objectives

Using TensorFlow backend.


In [2]:
def get_data():
    words = []
    with open("words.txt") as fi:
        line = fi.readline()
        while(len(line)!=0):
            datapoint = line.split(' ')
            phon = datapoint[1].split('.')
            phon[-1] = phon[-1][:-1]
            words.append([datapoint[0], phon])
            line = fi.readline()
            
    return(words)

In [3]:
data = get_data()

In [4]:
data[:5]

[['aback', ['AH', 'B', 'AE', 'K']],
 ['abandon', ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N']],
 ['abandoned', ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N', 'D']],
 ['abandoning', ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N', 'IH', 'NG']],
 ['abandons', ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N', 'Z']]]

In [5]:
Xraw = [k[0] for  k in data]
Yraw = [k[1] for  k in data]

In [6]:
Xraw[:5]

['aback', 'abandon', 'abandoned', 'abandoning', 'abandons']

In [7]:
Xrawlens = np.array([len(x) for x in Xraw])
Xrawlens.max(), Xrawlens.argmax(), Xraw[Xrawlens.argmax()]

(20, 22468, 'uncharacteristically')

In [8]:
Yraw[:5]

[['AH', 'B', 'AE', 'K'],
 ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N'],
 ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N', 'D'],
 ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N', 'IH', 'NG'],
 ['AH', 'B', 'AE', 'N', 'D', 'AH', 'N', 'Z']]

In [9]:
Yrawlens = np.array([len(y) for y in Yraw])
Yrawlens.max(), Yrawlens.argmax(), Yraw[Yrawlens.argmax()], Xraw[Yrawlens.argmax()]

(16,
 4046,
 ['K',
  'AA',
  'M',
  'P',
  'AA',
  'R',
  'T',
  'M',
  'EH',
  'N',
  'T',
  'AH',
  'L',
  'AY',
  'Z',
  'D'],
 'compartmentalized')

In [10]:
def word2tensor(word, wordmaxlen=24):
    # print(word)
    word = '`' + word + '{'
    word2tensor.dummy_char_vec = np.array([0]*28)
    wordvec_array = []
    padding = wordmaxlen - len(word)
    for eachcharacter in word:
        tempchar = word2tensor.dummy_char_vec.copy()
        tempchar[ord(eachcharacter) - ord('`')] = 1
        wordvec_array.append(tempchar)
        
    for i in range(padding):
        wordvec_array.append(word2tensor.dummy_char_vec.copy())
    return np.array(wordvec_array)

In [11]:
# def word2tensor(word, wordmaxlen=24):
#     # print(word)
#     word = '`' + word + '{'
#     wordvec_array = []
#     padding = wordmaxlen - len(word)
#     for eachcharacter in word:
#         tempchar = (ord(eachcharacter) - ord('`'))/27.0
#         wordvec_array.append(tempchar)
        
#     for i in range(padding):
#         wordvec_array.append((ord('{') - ord('`'))/27.0)
#     return np.array(wordvec_array)

In [12]:
result = word2tensor("word")

In [13]:
result, result.shape

(array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

### Converting words to tensor representations and save them for fast loading

In [14]:
words = np.array([word2tensor(w) for w in Xraw])
words.shape

(24167, 24, 28)

In [15]:
original_shape = words.shape
np.savez_compressed('words2vec.npz', words = words)

In [16]:
# data = np.load('words2vec.npz')
# words = data['words']

### Phonemes: There are 39 phonemes, as shown below:
`
AA     odd     AA D        |   AE      at      AE T      
AH      hut     HH AH T     |   AO      ought   AO T     
AW      cow     K AW        |   AY      hide    HH AY D  
B       be      B IY        |   CH      cheese  CH IY Z  
D       dee     D IY        |   DH      thee    DH IY    
EH      Ed      EH D        |   ER      hurt    HH ER T  
EY      ate     EY T        |   F       fee     F IY     
G       green   G R IY N    |   HH      he      HH IY    
IH      it      IH T        |   IY      eat     IY T     
JH      gee     JH IY       |   K       key     K IY     
L       lee     L IY        |   M       me      M IY     
N       knee    N IY        |   NG      ping    P IH NG  
OW      oat     OW T        |   OY      toy     T OY     
P       pee     P IY        |   R       read    R IY D   
S       sea     S IY        |   SH      she     SH IY    
T       tea     T IY        |   TH      theta   TH EY T AH 
UH      hood    HH UH D     |   UW      two     T UW     
V       vee     V IY        |   W       we      W IY     
Y       yield   Y IY L D    |   Z       zee     Z IY     
ZH      seizure S IY ZH ER  | 
`

In [17]:
phoneme_map = {
    '\s': 0, # Phoneme starting marker
    'AA': 1,
    'AE': 2,
    'AH': 3,
    'AO': 4,
    'AW': 5,
    'AY': 6,
    'B': 7,
    'CH': 8,
    'D': 9,
    'DH': 10,
    'EH': 11,
    'ER': 12,
    'EY': 13,
    'F': 14,
    'G': 15,
    'HH': 16,
    'IH': 17,
    'IY': 18,
    'JH': 19,
    'K': 20,
    'L': 21,
    'M': 22,
    'N': 23,
    'NG': 24,
    'OW': 25,
    'OY': 26,
    'P': 27,
    'R': 28,
    'S': 29,
    'SH': 30,
    'T': 31,
    'TH': 32,
    'UH': 33,
    'UW': 34,
    'V': 35,
    'W': 36,
    'Y': 37,
    'Z': 38,
    'ZH': 39,
    '\e': 40 # Phoneme termination marker
}

In [18]:
def phonemes2tensor(phonemes, phonemesmaxlen=20):
    # print(word)
    phonemes = ['\s'] + phonemes + ['\e']
    phonemes2tensor.dummy_phoneme_vec = np.array([0]*41) # dummy phonetic symbol as stop
    phonemesvec_array = []
    padding = phonemesmaxlen - len(phonemes)
    for eachphoneme in phonemes:
        tempphoneme = phonemes2tensor.dummy_phoneme_vec.copy()
        tempphoneme[phoneme_map[eachphoneme]] = 1
        phonemesvec_array.append(tempphoneme)
        
    for i in range(padding):
        phonemesvec_array.append(phonemes2tensor.dummy_phoneme_vec.copy())
    return np.array(phonemesvec_array)

In [19]:
# def phonemes2tensor(phonemes, phonemesmaxlen=20):
#     # print(word)
#     phonemes = ['\s'] + phonemes + ['\e']
#     phonemesvec_array = []
#     padding = phonemesmaxlen - len(phonemes)
#     for eachphoneme in phonemes:
#         tempphoneme = phoneme_map[eachphoneme]
#         phonemesvec_array.append(tempphoneme/40.0)
        
#     for i in range(padding):
#         phonemesvec_array.append(1.0)
#     return np.array(phonemesvec_array)

In [20]:
result = phonemes2tensor(Yraw[12])
result, result.shape

(array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      

In [21]:
phonemes = np.array([phonemes2tensor(p) for p in Yraw])
phonemes.shape

(24167, 20, 41)

In [22]:
original_shape = words.shape
np.savez_compressed('phonemes2vec.npz', phonemes = words)

In [23]:
# data = np.load('words2vec.npz')
# phonemes = data['phonemes']

In [24]:
timesteps = 24
_timesteps = 20
input_dim = 28
output_dim = 41
intermediate_dim = 64
latent_dim = 128
batch_size = 32
epsilon_std = 1.0

import tensorflow as tf

with tf.device("CPU:0"):
    x = keras.Input(shape=(timesteps, input_dim), name='input')

    # LSTM encoding
    h = keras.layers.LSTM((intermediate_dim), name='input_2_lstm1')(x)

    # VAE Z layer
    z_mean = keras.layers.Dense(latent_dim, name='lstm1_2_dense1')(h)
    z_log_sigma = keras.layers.Dense(latent_dim, name='dense1_2_dense2')(h)

    def sampling(args):
        z_mean, z_log_sigma = args
        epsilon = K.random_normal(shape=(batch_size, latent_dim),
                                  mean=0., stddev=epsilon_std)
        return z_mean + z_log_sigma * epsilon

    # note that "output_shape" isn't necessary with the TensorFlow backend
    # so you could write `Lambda(sampling)([z_mean, z_log_sigma])`
    z = keras.layers.Lambda(sampling, output_shape=(latent_dim,),name='dense2_2_lambda1')([z_mean, z_log_sigma])

    # decoded LSTM layer
    decoder_h = keras.layers.LSTM(intermediate_dim, return_sequences=True, name='dh_lstm')
    decoder_mean = keras.layers.LSTM(input_dim, return_sequences=True, name='du_lstm')

    h_decoded = keras.layers.RepeatVector(timesteps, name='lambda1_2_repeat')(z)
    h_decoded = decoder_h(h_decoded)

    # decoded layer
    x_decoded_mean = decoder_mean(h_decoded)

    # end-to-end autoencoder
    vae = keras.Model(x, x_decoded_mean)

    # encoder, from inputs to latent space
    encoder = keras.Model(x, z_mean)

    # generator, from latent space to reconstructed inputs
    decoder_input = keras.layers.Input(shape=(latent_dim,))

    _h_decoded = keras.layers.RepeatVector(timesteps)(decoder_input)
    _h_decoded = decoder_h(_h_decoded)

    _x_decoded_mean = decoder_mean(_h_decoded)
    generator = keras.Model(decoder_input, _x_decoded_mean)

    def vae_loss(x, x_decoded_mean):
        xent_loss = objectives.mse(x, x_decoded_mean)
        kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma))
        loss = xent_loss + kl_loss
        return loss

    vae.compile(optimizer='rmsprop', loss=vae_loss)

    vae.summary()


    vae.fit(x=words, y=words, epochs=2)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 24, 28)       0                                            
__________________________________________________________________________________________________
input_2_lstm1 (LSTM)            (None, 64)           23808       input[0][0]                      
__________________________________________________________________________________________________
lstm1_2_dense1 (Dense)          (None, 128)          8320        input_2_lstm1[0][0]              
__________________________________________________________________________________________________
dense1_2_dense2 (Dense)         (None, 128)          8320        input_2_lstm1[0][0]              
__________________________________________________________________________________________________
dense2_2_l

InvalidArgumentError: Incompatible shapes: [32] vs. [7]
	 [[Node: training/RMSprop/gradients/loss/du_lstm_loss/mul_1_grad/BroadcastGradientArgs = BroadcastGradientArgs[T=DT_INT32, _class=["loc:@training/RMSprop/gradients/loss/du_lstm_loss/mul_1_grad/Sum"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](training/RMSprop/gradients/loss/du_lstm_loss/mul_1_grad/Shape, training/RMSprop/gradients/loss/du_lstm_loss/mul_1_grad/Shape_1)]]

**** --It doesn't seems to make any progress. The process feels very arbitrary. Its just as bad as CNN technique.-- ****